## **基于卷积神经网络的结合位点预测**


#### **1. 导入所有需要的模块**

In [1]:
import os
import math
import torch
import random
import numpy as np
import pandas as pd
from torch import nn
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


#### **2. 设置全局的随机seed，确保结果的可复现性**

In [2]:
def all_seed(seed = 24):
    np.random.seed(seed)
    random.seed(seed)
    # CPU
    torch.manual_seed(seed) 
    # GPU
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed) 
    # python 全局
    os.environ['PYTHONHASHSEED'] = str(seed) 
    # cudnn
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    print(f'Set env random_seed = {seed}')

    print('全局随机设置已完成')

#### **3. 定义预处理文件的函数**

In [3]:
def trans_seq(seq):
    trans_result = []
    encoder = {
        'A': np.array([1, 0, 0, 0]),
        'C': np.array([0, 1, 0, 0]),
        'G': np.array([0, 0, 1, 0]),
        'T': np.array([0, 0, 0, 1]),
        'N': np.array([0.25, 0.25, 0.25, 0.25])
    }
    for base in seq:
        trans_result.append(encoder[base])
    return np.array(trans_result)

def process_csv(filename):
    df = pd.read_csv(filename)
    df.head()
    seqs = df['seq'].to_list()
    seqs = np.array([list(seq) for seq in seqs])
    results = df['result'].to_list()
    results = np.array([1 if result == True else 0 for result in results])
    trans_seqs = np.apply_along_axis(trans_seq, 1, seqs)
    seq_tensor = torch.tensor(trans_seqs, dtype=torch.float32)
    seq_tensor_trans = seq_tensor.transpose(1, 2)
    results_tensor = torch.tensor(results, dtype=torch.float32)
    
    # 序列的张量维度为(num_of_samples, input_channels, seq_length)，这里的input_channels = 4，对应其中N使用了四个位置的均等编码方式
    # 结果的张量维度为(num_of_samples, )
    return seq_tensor_trans, results_tensor 


#### **4. 定义全局的参数字典，可以在我们后续调参过程中提供很大的方便**

In [4]:
config = {
    'batch_size': 64,
    'kernel_size': 10,
    'num_epochs': 50,
    'padding_size':0,
    'learning_rate':0.001,
    'random_seed': 24,
    'weight_decay':1e-4,
    'dropout_probability_cnn':0.2,
    'dropout_probability_linear':0.3,
    'save_path': './models/model.ckpt'
}

#### **5. 搭建卷积神经网络和后续的全连接层**

对于序列这种一维的数据，我们考虑使用1D卷积，这种卷积方式卷积核的移动是一维的，输入数据应该被整合为[`batch_size, input_channel, n_feature`]的格式  

1D卷积的输出长度公式如下：
$$
\text{Output Length} = \left\lfloor \frac{\text{Input Length} - \text{Kernel Size} + 2 \times \text{Padding}}{\text{Stride}} \right\rfloor + 1
$$


In [5]:
import torch
import torch.nn as nn

class DNA_CNN(nn.Module):
    def __init__(self):
        super(DNA_CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(in_channels=4, out_channels=32, kernel_size=config['kernel_size'], stride=1, padding=config['padding_size']),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=config['kernel_size'], stride=1, padding=config['padding_size']),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=config['kernel_size'], stride=1, padding=config['padding_size']),
            nn.ReLU(inplace=True),

            nn.AdaptiveMaxPool1d(20) # 自适应的池化层
        )
        
        self.full_connection = nn.Sequential(
            nn.Linear(128*20, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        output = self.full_connection(x)
        return output
     

#### **6. 建立数据集和数据加载器**

In [7]:
'''
为了在训练中评估模型的性能，我将一部分的训练数据划分为validation dataset，并在每个epoch结束后评估模型在这个数据集上的表现。
'''

X_train, y_train = process_csv('K562_CTCF_train_seq_result.csv')
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=config['random_seed']) # 提取0.2的训练数据作为验证集
print('训练数据规模:',X_train.shape)
print('验证数据规模:',X_valid.shape)

X_test, y_test = process_csv('K562_CTCF_test_seq_result.csv')
print('测试数据规模:',X_test.shape)

train_data = TensorDataset(X_train, y_train)
valid_data = TensorDataset(X_valid, y_valid)
test_data = TensorDataset(X_test, y_test)

'''
创建DataLoader
'''
train_loader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=config['batch_size'], shuffle=False)
test_loader = DataLoader(test_data, batch_size=config['batch_size'], shuffle=False)

训练数据规模: torch.Size([6400, 4, 1000])
验证数据规模: torch.Size([1600, 4, 1000])
测试数据规模: torch.Size([2000, 4, 1000])


#### **7. 模型训练**

In [8]:
def trainer(train_loader, valid_loader, model, config, device):
    criterion = nn.BCELoss() 
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])

    # 创建保存模型的目录
    if not os.path.isdir('./models'):
        os.mkdir('./models')
    
    save_path = config['save_path']
    best_loss = math.inf
    best_accuracy = 0

    for epoch in range(config['num_epochs']):
        model.train() 
        train_loss_record = []
        train_accs = []
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for inputs, labels in train_pbar:
            optimizer.zero_grad()  
            inputs, labels = inputs.to(device), labels.to(device) 

            outputs = model(inputs)
            outputs = outputs.squeeze()  
            loss = criterion(outputs, labels.float()) 

            loss.backward() 
            if config.get('clip_flag', False):
                nn.utils.clip_grad_norm_(model.parameters(), max_norm=10) 
            optimizer.step() 

            acc = ((outputs > 0.5).float() == labels).float().mean()  # 计算准确率
            train_loss_record.append(loss.item())
            train_accs.append(acc.item())

            train_pbar.set_description(f'Epoch [{epoch+1}/{config["num_epochs"]}]')
            train_pbar.set_postfix({'loss': f'{loss.item():.5f}', 'acc': f'{acc.item():.5f}'})
        
        mean_train_acc = sum(train_accs) / len(train_accs)
        mean_train_loss = sum(train_loss_record) / len(train_loss_record)

        # 验证集评估
        model.eval()  
        valid_loss_record = []
        valid_accs = []
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                outputs = outputs.squeeze()
                loss = criterion(outputs, labels.float())
                acc = ((outputs > 0.5).float() == labels).float().mean()

                valid_loss_record.append(loss.item())
                valid_accs.append(acc.item())
        
        mean_valid_acc = sum(valid_accs) / len(valid_accs)
        mean_valid_loss = sum(valid_loss_record) / len(valid_loss_record)

        print(f'Epoch [{epoch+1}/{config["num_epochs"]}]: Train loss: {mean_train_loss:.4f}, acc: {mean_train_acc:.4f}, Valid loss: {mean_valid_loss:.4f}, acc: {mean_valid_acc:.4f}')

        if mean_valid_loss < best_loss:
            best_accuracy = mean_valid_acc
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), save_path)
            print(f'Saving model with loss {best_loss:.4f} and accuracy {best_accuracy:.4f}...')
            


def main():

    all_seed(config['random_seed'])
    
    # 使用笔记本显卡加速训练
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = DNA_CNN().to(device)

    trainer(train_loader, valid_loader, model, config, device)

main()

Set env random_seed = 24
全局随机设置已完成


Epoch [1/50]: 100%|██████████| 100/100 [00:02<00:00, 45.96it/s, loss=0.68916, acc=0.57812]


Epoch [1/50]: Train loss: 0.6923, acc: 0.5244, Valid loss: 0.6936, acc: 0.5044
Saving model with loss 0.6936 and accuracy 0.5044...


Epoch [2/50]: 100%|██████████| 100/100 [00:01<00:00, 52.01it/s, loss=0.64233, acc=0.62500]


Epoch [2/50]: Train loss: 0.6648, acc: 0.5816, Valid loss: 0.6602, acc: 0.6069
Saving model with loss 0.6602 and accuracy 0.6069...


...

Epoch [48/50]: Train loss: 0.0005, acc: 1.0000, Valid loss: 2.0416, acc: 0.8075


Epoch [49/50]: 100%|██████████| 100/100 [00:01<00:00, 53.18it/s, loss=0.00091, acc=1.00000]


Epoch [49/50]: Train loss: 0.0005, acc: 1.0000, Valid loss: 2.0391, acc: 0.8063


Epoch [50/50]: 100%|██████████| 100/100 [00:01<00:00, 52.25it/s, loss=0.00046, acc=1.00000]


Epoch [50/50]: Train loss: 0.0005, acc: 1.0000, Valid loss: 2.0318, acc: 0.8081


#### **8. 测试集上来评估模型的效能**

In [9]:
# 加载保存的最佳模型
def load_best_model(model, save_path, device):
    model.load_state_dict(torch.load(save_path))
    model.to(device)  
    model.eval()  # 切换为评估模式
    print(f"Loaded model weights from {save_path}")
    return model

# 使用保存的模型进行测试评估
def evaluate_on_test_set(test_loader, model, device):
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            predicted = (outputs.squeeze() > 0.5).float()
            
            all_preds.extend(predicted.cpu().numpy()) 
            all_labels.extend(labels.cpu().numpy()) 
    
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'测试集准确率: {accuracy * 100:.2f}%')
    
    report = classification_report(all_labels, all_preds)
    print("分类报告:")
    print(report)

model = DNA_CNN()  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载保存的最佳模型权重
model = load_best_model(model, config['save_path'], device)

# 使用加载的模型评估测试集
evaluate_on_test_set(test_loader, model, device)


Loaded model weights from ./models/model.ckpt
测试集准确率: 80.80%
分类报告:
              precision    recall  f1-score   support

         0.0       0.79      0.83      0.81       979
         1.0       0.83      0.79      0.81      1021

    accuracy                           0.81      2000
   macro avg       0.81      0.81      0.81      2000
weighted avg       0.81      0.81      0.81      2000



**可以看到，配合卷积神经网络之后，模型的预测能力有了很大的提升，整体准确率可以达到81%左右，值得一提的是，对于网络的结构调整以及一些超参数调整，对于模型性能的提升不大，个人认为模型对于这个不复杂的二分类任务还是相对较大的，准确率不能再提升的原因可能是数据编码的策略/数据集的质量问题。**